**Blob Inventory Analytics Tool**

# Copyright (c) Microsoft Corporation.
# Licensed under the MIT License.

In [ ]:
# set the details of account and container where BlobInventoryStorageAccountConfiguration.json file is stored and the storage account 
# is connected to synapse workspace
storage_account = "DataLakeStorageGen2Account"
container_name = "DataLakeStorageGen2Container"
file_name = "BlobInventoryStorageAccountConfiguration"

# name of the database in which tables will be stored
database_name = "reportdata"

In [ ]:
# uninstalling current version as BlobService is disabled in current version
%pip uninstall azure-storage-blob --yes
# installing previous version as BlobService is available in this version
%pip install azure-storage-blob==2.1.0

In [ ]:
# importing libraries
from pyspark.sql import *
from pyspark.sql.types import *
from pyspark.sql.functions import *
from pyspark.sql.functions import count as count
from pyspark.sql.functions import substring_index
from pyspark.sql.functions import lit
from pyspark.sql.functions import col
from azure.storage.blob import BlockBlobService
import datetime
import sys

In [ ]:
# storing distribution of blobs in different containers 
def store_data_distribution_in_containers_details(csvFile, database_name):
    # extracting container name from the 'Name' field
    container_info_df = csvFile.withColumn("ContainerName", substring_index(csvFile.Name, '/', 1))
    # grouping dataframe on the basis of report generation date and then on the basis of container name
    container_info_df = container_info_df.groupBy(['ReportGenerationDate','ContainerName']).sum('Content-Length').withColumnRenamed("sum(Content-Length)", "Size")
    container_info_df.write.mode('ignore').saveAsTable("{0}.ContainerInfo".format(database_name))

In [ ]:
# storing table containing blobtype and their corresponding count with respect to every inventory report analysed ( i.e report generation date)
def store_data_distribution_in_blob_details(csvFile, database_name):
    # grouping dataframe on the basis of report generation date and then further on the basis of Blob Type
    blob_type_info_df = csvFile.groupBy(['ReportGenerationDate','BlobType']).agg({'Content-Length':'sum'}).withColumnRenamed("sum(Content-Length)", "Size")
    blob_type_info_df.write.mode('ignore').saveAsTable("{0}.BlobTypeInfo".format(database_name))

In [ ]:
# storing the distribution of blobs in different access tiers
def store_data_distribution_in_access_tier_details(csvFile, database_name):
    # filtering dataframe that are block blob and then grouping on the basis of report generation date and further on the basis of access tier to which that blob belong
    access_tier_info_df = csvFile.filter(csvFile.BlobType=="BlockBlob").groupBy(['ReportGenerationDate','AccessTier']).agg({'Content-Length':'sum'}).withColumnRenamed("sum(Content-Length)", "Size")
    access_tier_info_df.write.mode('ignore').saveAsTable("{0}.AccessTierInfo".format(database_name))

In [ ]:
# storing soft deleted data size corresponding to the inventory report analysed
def store_soft_deleted_data_size_details(csvFile, database_name):
    if "Deleted" in csvFile.columns:
        # filtering dataframe where Deleted column is set to True and then aggregating the Content-Length for those rows
        soft_deleted_info_df = csvFile.filter(csvFile.Deleted==True)
        # if there are is no soft deleted data, will fill the table with default value i.e 0 size
        if soft_deleted_info_df.rdd.isEmpty():
            soft_deleted_info_df = csvFile.groupBy("ReportGenerationDate").agg({'Content-Length':'sum'}).withColumnRenamed("sum(Content-Length)", "Size")
            soft_deleted_info_df = soft_deleted_info_df.withColumn("Size",lit(0))
        else:
            soft_deleted_info_df = soft_deleted_info_df.groupBy("ReportGenerationDate").agg({'Content-Length':'sum'}).withColumnRenamed("sum(Content-Length)", "Size")
    else:
        # filling the table wih default values ( i.e Size is 0 ) if 'Deleted' Column is not present
        soft_deleted_info_df = csvFile.groupBy("ReportGenerationDate").agg({'Content-Length':'sum'}).withColumnRenamed("sum(Content-Length)", "Size")
        soft_deleted_info_df = soft_deleted_info_df.withColumn("Size",lit(0))
    soft_deleted_info_df.write.mode('ignore').saveAsTable("{0}.SoftDeletedInfo".format(database_name))

In [ ]:
# storing the distribution of blob corresponding to the type of file ( ex - pdf, json, png )
def store_content_type_distribution_details(csvFile, database_name):
    # handling folder as a separate type of file
    # filtering the dataframe for selecting folders by checking 'hdi_isfolder' column
    if "hdi_isfolder" in csvFile.columns:
        content_type_info_df = csvFile.withColumn("FileType", when(csvFile['hdi_isfolder']==True,'Unknown').otherwise(substring_index(csvFile['Name'], '.', -1)))
    else:
        # selecting the right side of the last '.' in the Name column as the content-type of blob
        content_type_info_df = csvFile.withColumn("FileType", when(csvFile['Name'].contains("."),substring_index(csvFile['Name'], '.', -1)).otherwise('Unknown'))
    # grouping the dataframe on the basis of report generation date and further on the basis of type of file
    content_type_info_by_count_df = content_type_info_df.groupBy(['ReportGenerationDate','FileType']).agg(sum('Content-Length').alias('Sum'))
    content_type_info_by_count_df.write.mode('ignore').saveAsTable("{0}.ContentTypeInfo".format(database_name))

In [ ]:
# storing Content-Length of the data corresponding to the report being analysed
# it tells the growth in the size of data in the storage account
def store_data_growth_in_account_details(csvFile, database_name):
    # creating dataframe and storing table containing Date and corresponding data size on that day 
    # by grouping the data on the basis of report generation date and then aggregating the Content-Length
    growth_in_data_df = csvFile.groupBy('ReportGenerationDate').sum('Content-Length').withColumnRenamed("sum(Content-Length)", "Size").orderBy("ReportGenerationDate","Size")
    growth_in_data_df.write.mode('ignore').saveAsTable("{0}.dailyDataSizeInfo".format(database_name))

In [ ]:
# storing the amount of data created on respective date
def store_data_creation_in_account_details(csvFile, database_name):
    # converting the 'Creation-Time' column to date type
    data_creation_with_time_df = csvFile.withColumn("Date", to_date(csvFile['Creation-Time'], 'dd-MM-yyyy'))
    # grouping the dataframe on the basis of 'Creation-Date' and aggregating the Content-Length
    data_creation_with_time_df = data_creation_with_time_df.groupBy('Date').sum('Content-Length').withColumnRenamed("sum(Content-Length)", "Size")
    data_creation_with_time_df.write.mode('ignore').saveAsTable("{0}.dataCreationWithTime".format(database_name))

In [ ]:
def store_last_access_time_details_for_df(last_access_time_df, table_name, max_ts, min_ts, database_name, csvFile):
    # filtering the dataframe on the basis of the last report analysed (date basis) as we will only use last report to get the LastAccessTime
    last_access_time_df = last_access_time_df.filter(last_access_time_df.ReportGenerationDate==max_ts)
    if "LastAccessTime" in csvFile.columns:
        # changing datatype of column LastAccessTime to Date type
        last_access_time_df = last_access_time_df.withColumn("LastAccessTime", to_date('LastAccessTime'))
        # if LastAccessTime is null or empty, will fill it with the total span of days for which the reports are generated calcuated by subtracting current date minus the date of first report generated(current date  - min_ts)
        last_access_time_df = last_access_time_df.withColumn("DaysLastAccessed", when((last_access_time_df["LastAccessTime"].isNull() | (last_access_time_df["LastAccessTime"]=='')),(datetime.datetime.now().date()-min_ts).days).otherwise(((unix_timestamp(current_date(),"dd") - unix_timestamp(last_access_time_df["LastAccessTime"], "dd"))/86400).cast(IntegerType())))
    else:
        # using default value for 'DaysLastAccessed' as the number of days between current date and first generated report date
        last_access_time_df = last_access_time_df.withColumn("DaysLastAccessed", (datetime.datetime.now().date()-min_ts).days)
    # creating a column 'TotalSize' that stores the total amount of data present in hot tier and is block blob
    last_access_time_df = last_access_time_df.withColumn("TotalSize",lit(last_access_time_df.agg({"Content-Length":"sum"}).collect()[0][0]))
    # if the dataframe is empty, then will fill with default values
    if last_access_time_df.rdd.isEmpty():
        last_access_time_df = spark.createDataFrame([Row(max_ts,"Name1",0,0,0),Row(min_ts,"Name2",0,(datetime.datetime.now().date()-min_ts).days,0)],["ReportGenerationDate","Name","Content-Length","DaysLastAccessed","TotalSize"])
    last_access_time_df.select("ReportGenerationDate","Name","Content-Length","DaysLastAccessed","TotalSize").write.mode('ignore').saveAsTable("{0}.{1}".format(database_name, table_name))

# storing the Name, Content-Length of the file along with the days for which it is not accessed (i.e Current Date minus the LastAccessTime)
def store_last_access_time_details(csvFile, database_name):
    
    # filtering the dataframe whether the data is in Block blob
    last_access_time_df = csvFile.filter(csvFile.BlobType=="BlockBlob")
    # storing the date of the last inventory report analysed
    max_ts = csvFile.agg({"ReportGenerationDate": "max"}).collect()[0][0]
    # storing the date to the first inventory report analysed
    min_ts = csvFile.agg({"ReportGenerationDate": "min"}).collect()[0][0]

    last_access_time_hot_tier_df = last_access_time_df.filter(last_access_time_df.AccessTier=="Hot")
    last_access_time_cool_tier_df = last_access_time_df.filter(last_access_time_df.AccessTier=="Cool")

    store_last_access_time_details_for_df(last_access_time_hot_tier_df, "lastAccessTimeHotTier", max_ts, min_ts, database_name, csvFile)
    store_last_access_time_details_for_df(last_access_time_cool_tier_df, "lastAccessTimeCoolTier", max_ts, min_ts, database_name, csvFile)

In [ ]:
# storing the size of data occupied by snapshots
def store_data_occupied_by_snapshot_details(csvFile, database_name):
    # checking whether 'Snapshot' field is present in the inventory report(dataframe)
    if "Snapshot" in csvFile.columns:
        # dropping all the rows having null values in Snapshot column
        snapshot_data_df = csvFile.na.drop(how='all',subset=['Snapshot'])
        if snapshot_data_df.rdd.isEmpty():
            snapshot_data_df = csvFile.groupBy("ReportGenerationDate").agg({'Content-Length':'sum'}).withColumnRenamed("sum(Content-Length)", "Size")
            # using default value of Snapshot size as 0 as 'Snapshot' field is disabled
            snapshot_data_df = snapshot_data_df.withColumn("Size", lit(0))
        else:
            # grouping dataframe on the basis of report generation date and then aggregating the Content-Length
            snapshot_data_df = snapshot_data_df.groupBy("ReportGenerationDate").agg({'Content-Length':'sum'}).withColumnRenamed("sum(Content-Length)", "Size")
    else:
        snapshot_data_df = csvFile.groupBy("ReportGenerationDate").agg({'Content-Length':'sum'}).withColumnRenamed("sum(Content-Length)", "Size")
        # using default value of Snapshot size as 0 as 'Snapshot' field is disabled
        snapshot_data_df = snapshot_data_df.withColumn("Size", lit(0))
    snapshot_data_df.write.mode('ignore').saveAsTable("{0}.snapshotData".format(database_name))

In [ ]:
# storing number of modifications done corresponding to the respective date
def store_modifications_in_data_details(csvFile, database_name):
    #checking whether Last-Modified field is present in the inventory report
    if "Last-Modified" in csvFile.columns:
        last_modified_count_df = csvFile.withColumn("Date", to_date(csvFile['Last-Modified'], 'dd-MM-yyyy'))
        last_modified_count_df = last_modified_count_df.groupBy('Date').agg(count('Date').alias('NumberOfModifications'))
    else:
        last_modified_count_df = csvFile.groupBy('ReportGenerationDate').agg(count('ReportGenerationDate').alias('NumberOfModifications'))
        # using default value as 0 to show that 'Last-Modified' field is not present and we need entry for every report date analysed otherwise it will hamper the relationship between tables
        last_modified_count_df = last_modified_count_df.withColumn("NumberOfModifications", lit(0))
    last_modified_count_df.write.mode('ignore').saveAsTable("{0}.lastModifiedCount".format(database_name))

In [ ]:
# storing the dates i.e report generation date of all the reports analysed
def store_reports_analysed_dates(csvFile, database_name):
    reports_analysed_df = csvFile.select("ReportGenerationDate").distinct()
    reports_analysed_df.write.mode('ignore').saveAsTable("{0}.reportsanalysed".format(database_name))

In [ ]:
# returns all the json file links to the respective inventory reports (by providing the destination container and the rule name along with storage account credentials)
def get_json_link_of_reports(storage_account_name, access_key, destination_container, rule_name):
    blob_service = BlockBlobService(storage_account_name, access_key)
    # list the relative path to all the blobs present in the destination container
    try:
        blob_list = blob_service.list_blobs(destination_container)
    except Exception as e:
        print("Error: Container does not exist", e)
        return
    # storing the links to all the blob inventory reports
    links_list = []
    # iterating over the returned list of relative path to blobs
    for blob in blob_list:
        # checking if the relative path contains 'ruleName-manifest.json' and correspondingly creating a link to that json file
        if rule_name+"-manifest.json" in blob.name:
            link = "wasbs://{0}@{1}.blob.core.windows.net/{2}".format(destination_container,storage_account_name,blob.name)
            links_list.append(link)
    return links_list

In [ ]:
# calling functions for respective processing and storing them in the form of tables
# every table corresponds to respective visualization
def populating_tables(csvFile, database_name):
    # checking if the dataframe is empty that is no data to process
    if csvFile.rdd.isEmpty():
        return
    store_data_distribution_in_containers_details(csvFile, database_name)
    store_data_distribution_in_blob_details(csvFile, database_name)
    store_data_distribution_in_access_tier_details(csvFile, database_name)
    store_soft_deleted_data_size_details(csvFile, database_name)
    store_content_type_distribution_details(csvFile, database_name)
    store_data_growth_in_account_details(csvFile, database_name)
    store_data_creation_in_account_details(csvFile, database_name)
    store_last_access_time_details(csvFile, database_name)
    store_data_occupied_by_snapshot_details(csvFile, database_name)
    store_modifications_in_data_details(csvFile, database_name)
    store_reports_analysed_dates(csvFile, database_name)

In [ ]:
# setting the credentials of the spark session
def setting_credentials(storage_account_name, access_key):
    access_link = "fs.azure.account.key.{0}.blob.core.windows.net".format(storage_account_name)
    try:
        spark.conf.set(access_link,access_key)
    except Exception as e:
        print("Error: Unable to set credentials for spark session", e)

In [ ]:
# processing multiple inventory reports by processing their respective json file and adding them to the dataframe
def processing_reports(list_of_report_links_json_file, report_dates_analysed):
    
    reports_analysed = []
    fileData = None
    # iterating over rest of the the rows
    for json_file_report_link in list_of_report_links_json_file:
        try:
            report_df = spark.read.option("multiLine", "true").json(json_file_report_link)
        except Exception as e:
            print("Error: Unable to access Blob Inventory Report Json File", e)
            return

        try:
            data_collect = report_df.collect()[0]
        except Exception as e:
            print("Error: Unable to read Blob Inventory Report Json File", e)
            return

        # skip if the report is pending
        if(data_collect["status"]=='Pending'):
            continue

        destination_container = data_collect["destinationContainer"]
        report_generation_date = data_collect["inventoryCompletionTime"].split("T")[0]
        file_format = data_collect["ruleDefinition"]["format"]
      
        # skip if the report if already analysed
        if report_generation_date in reports_analysed or report_generation_date in report_dates_analysed:
            continue

        if (len(data_collect["files"]) == 0):
            continue

        fileDataTemp = None
        for partitioned_file in data_collect["files"]:
            prefix_path = data_collect["files"][0]["blob"]
            fileLink = "wasbs://{0}@{1}.blob.core.windows.net/{2}".format(destination_container, storage_account_name, prefix_path)

            # checking whether the report is in csv or parquet format
            try:
                if file_format=="csv":
                    partitionedFileDataTemp = spark.read.csv(fileLink, header=True, inferSchema=True)
                else:
                    partitionedFileDataTemp = spark.read.parquet(fileLink)
            except Exception as e:
                print("Error: Unable to access Blob Inventory Reports", e)
                return

            if fileDataTemp is None:
                fileDataTemp = partitionedFileDataTemp
            else:
                fileDataTemp = fileDataTemp.unionByName(partitionedFileDataTemp, allowMissingColumns=True)

        fileDataTemp = fileDataTemp.withColumn('ReportGenerationDate',lit(report_generation_date))
        reports_analysed.append(report_generation_date)
        try:
            if fileData is None:
                fileData = fileDataTemp
            else:
                fileData = fileData.unionByName(fileDataTemp, allowMissingColumns=True)
        except Exception as e:
            print("Error: Blob Inventory Reports have mis-matching fields", e)
            return
    
    if fileData is None:
        return None
        
    # changing datatype of column Content-Length from string to integer
    try:
        fileData = fileData.withColumn("Content-Length", fileData["Content-Length"].cast(IntegerType()))
    except Exception as e:
        print("Error: Content-Length field is missing from the inventory reports", e)
    # changing datatype of column ReportGenerationDate to Date type
    fileData = fileData.withColumn("ReportGenerationDate", to_date('ReportGenerationDate'))
    
    return fileData

In [ ]:
# processing and returning the configuration file data
def get_configuration_file_data(storage_account, container_name, file_name):
    configuration_file_link = "abfss://{0}@{1}.dfs.core.windows.net/{2}.json".format(container_name, storage_account, file_name)
    try:
        configuration_file = spark.read.option("multiLine", "true").json(configuration_file_link)
    except Exception as e:
        print("Error: Unable to access Blob Inventory Storage Account Configuration File", e)
        return
    try:
        configuration_file_data = configuration_file.collect()[0]
    except Exception as e:
        print("Error: Unable to read Blob Inventory Storage Account Configuration File", e)
        return
    return configuration_file_data

In [ ]:
# returns the list of all the reports that are analysed till now by returning the data stored in the table 'reportsanalysed'
def get_report_dates(database_name):
    # creating database if it does not exist
    spark.sql("CREATE DATABASE IF NOT EXISTS {0}".format(database_name))
    # retrieving al the tables stored in the database
    table_list=spark.sql("SHOW TABLES IN {0}".format(database_name))
    # filtering whether 'reportsanalysed' table is present in the database 
    table_data=table_list.filter(table_list.tableName=="reportsanalysed").collect()
    # list of all the reports analysed 
    table_res_list = []
    if len(table_data)>0:
        # fetching the 'reportsanalysed' table
        fetch_table = "SELECT * FROM {0}.reportsanalysed".format(database_name)
        table_res = spark.sql(fetch_table)
        # changing datatype of column ReportGenerationDate from DateType to StringType
        table_res = table_res.withColumn("ReportGenerationDate", table_res["ReportGenerationDate"].cast(StringType()))
        # converting the dataframe to the list type
        table_res_list = table_res.select("ReportGenerationDate").rdd.flatMap(lambda x: x).collect()
    return table_res_list

In [ ]:
# reading the data from template file
json_file_data = get_configuration_file_data(storage_account, container_name, file_name)

# initializing with the data stored in template file
try:
    storage_account_name = json_file_data["storageAccountName"]
    access_key = json_file_data["accessKey"]
    destination_container = json_file_data["destinationContainer"]
    rule_name = json_file_data["blobInventoryRuleName"]
except Exception as e:
    print("Error: Invalid Format of Blob Inventory Storage Account Configuration File", e)
    mssparkutils.notebook.exit("0")

# setting credentials for the spark session
setting_credentials(storage_account_name, access_key)

# list of all the report links
list_of_report_links = get_json_link_of_reports(storage_account_name, access_key, destination_container, rule_name)

if(len(list_of_report_links)!=0):
    # retreving all the reports analysed till now
    report_dates_analysed = get_report_dates(database_name)

    # processsing all the reports in a dataframe
    file_data = processing_reports(list_of_report_links, report_dates_analysed)
    if file_data is None:
        print("Nothing to process. Exiting...")
        mssparkutils.notebook.exit("1")

    # storing all processed dataframes in respective tables
    populating_tables(file_data, database_name)

In [ ]:
def cleanup():
    database_name = "reportdata"

    # command to drop the database and the corresponding tables
    spark.sql("DROP DATABASE IF EXISTS {0} CASCADE".format(database_name))

    # command to iterate over a table
    table_name = "blobtypeinfo"
    spark.sql('SELECT * from {0}.{1}'.format(database_name, table_name)).show()
